# Notas Práctico 1 - Marcelo Huerta San Martín

In [ ]:
# Obviamente la ruta varía según la instalación local
rutacsv = '~/7506/data/fiuba-trocafone-tp1-final-set/events.csv'

In [ ]:
import pandas as pd
troca = pd.read_csv(rutacsv, 
    low_memory=False,
    infer_datetime_format=True,
    parse_dates=['timestamp'],
    na_values=['Unknown'],   # países desconocidos se cambian a NA
)

In [ ]:
# Exploración inicial para determinar qué eventos son los que contienen información de país, región y ciudad para 
# una persona
cualpais = troca.loc[:, ['event', 'person', 'country']].dropna()
cualpais['event'].drop_duplicates()

In [ ]:
# De lo anterior, solo 'visited site' contiene datos de país, se crea el df en consecuencia
# se remueven aquellas personas que tengan país NA (por haber sido originalmente Unknown)
pais_por_persona = troca.loc[troca['event'] == 'visited site', ['person', 'country']].dropna(axis=0, how='any')
pais_por_persona.set_index('person', inplace=True)
# Este dataframe luego podrá ser usado en otras búsquedas

## Exploraciones preliminares

### Países con más accesos

In [ ]:
ev_por_persona = troca.loc[:, ['event', 'person']]
ev_por_pais = (
    ev_por_persona.merge(pais_por_persona, on='person', how='left') # Agregar el país al df
    .groupby(['country', 'event'])['country']
    .count()
    .unstack('event').fillna(0)
)

_ = ev_por_pais.plot(kind='bar', stacked='true', logy=True, figsize=(20,15))
_.set_title('Eventos por país de origen', fontsize=24)
_.set_xlabel('País', fontsize=18)
_.set_ylabel('Cantidad de eventos (escala log.)', fontsize=18)

### Impacto publicitario

In [ ]:
# Columnas relevantes en los eventos de impacto publicitario?
troca[troca.event == 'ad campaign hit'].dropna(axis=1, how='all').drop('event', axis=1).head()

In [ ]:
# Se toman en cuenta todos los hits por campaña considerando sólo persona y origen de la campaña, 
# y luego se asocia al país por la persona (descartando aquellos donde el país de la persona es desconocido)
hits = (
    troca.loc[troca['event'] == 'ad campaign hit', ['person', 'campaign_source']]
    .merge(pais_por_persona, on='person', how='left')
    .dropna(how='any')
).set_index('country')

# Elegimos los países más significativos
paises = ['Argentina', 'Brazil', 'United States']
hits_signif = hits.loc[paises, :].reset_index()

# Los agrupamos por campaña y país, contándolos y luego desagregando por país para graficar

hits_por_pais = (hits_signif.groupby(['campaign_source', 'country', ])
     .count()
     .unstack('country')
     .fillna(0)
)
hits_por_pais.columns = hits_por_pais.columns.get_level_values(1)
hits_por_pais = hits_por_pais.T

_ = hits_por_pais.plot(kind='bar', figsize=(20,10), logy=True, grid=True)
_.set_title('Impacto publicitario en los 3 países más significativos', fontsize=24)
_.set_ylabel('Cantidad de hits (escala log.)', fontsize=20)
_.set_xlabel('País', fontsize=20)

### Búsqueda de eventos de aquellas personas que hayan completado conversiones

In [ ]:
conversiones = troca.loc[troca.event == 'conversion', 'person'].drop_duplicates()
checkouts = troca.loc[troca.event == 'checkout', 'person'].drop_duplicates()
eventos_de_personas_con_conversiones = troca.loc[troca.person.isin(conversiones), :]
eventos_de_personas_con_conversiones